In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import random

# 使用 OpenAI GPT-4o 作為主要 LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- 分類客服 ---
classify_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是銀行客服人員，負責將使用者問題轉交正確部門。\n\n"
     "【客服文件】\n"
     "1. 帳務部門：處理存款、提款、餘額查詢、轉帳。\n"
     "2. 信用卡部門：處理掛失、刷卡爭議、額度調整。\n"
     "3. 若問題不屬於以上範圍，請回覆 'human'。\n\n"
     "請只輸出以下其中一個詞：'accounting'、'creditcard' 或 'human'。"),
    ("human", "{question}")
])
def classify(question: str) -> str:
    response = llm.invoke(classify_prompt.format_messages(question=question))
    raw = response.content.strip().lower()
    if "accounting" in raw:
        return "accounting"
    elif "creditcard" in raw:
        return "creditcard"
    else:
        return "human"

# --- 部門回答 ---
def accounting_answer() -> str:
    return "您好，我是帳務部門人員，正在處理您的帳務問題。"

def creditcard_answer() -> str:
    return "您好，我是信用卡部門人員，正在處理您的信用卡問題。"

# --- 信心度檢查 (隨機產生) ---
def confidence_check(answer: str) -> float:
    score = round(random.uniform(0, 1), 2)
    print(f"[信心度檢查] 答案: {answer} → 信心度={score:.2f}")
    return score

# --- 人工客服 ---
def human_fallback(question: str, retries: int) -> str:
    if retries == 0:
        return "AI客服無法為您解決問題，已轉人工客服處理。"
    else:
        return f"AI客服多次嘗試回答後仍信心不足，已轉人工客服處理。"

# --- LangChain 流程 ---
def customer_service(question: str, max_retries: int = 3) -> str:
    retries = 0
    while retries <= max_retries:
        route = classify(question)
        print(f"[分類] 問題: {question} → 部門: {route} (重試次數={retries})")

        if route == "human":
            return human_fallback(question, retries)

        # 取得部門回答
        if route == "accounting":
            answer = accounting_answer()
        elif route == "creditcard":
            answer = creditcard_answer()
        else:
            answer = human_fallback(question, retries)
            return answer

        # 檢查信心度
        confidence = confidence_check(answer)
        if confidence >= 0.5:
            return answer
        else:
            retries += 1
            if retries > max_retries:
                return human_fallback(question, retries)

In [5]:
# --- 測試1 ---
print(customer_service("請問為什麼我的轉帳失敗了？"))

[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=0)
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.63
[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=1)
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.68
[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=2)
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.28
[分類] 問題: 請問為什麼我的轉帳失敗了？ → 部門: accounting (重試次數=3)
[信心度檢查] 答案: 您好，我是帳務部門人員，正在處理您的帳務問題。 → 信心度=0.28
AI客服多次嘗試回答後仍信心不足，已轉人工客服處理。


In [6]:
# --- 測試2 ---
print(customer_service("我的信用卡昨天被盜刷了怎麼辦？"))

[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=0)
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.33
[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=1)
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.18
[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=2)
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.05
[分類] 問題: 我的信用卡昨天被盜刷了怎麼辦？ → 部門: creditcard (重試次數=3)
[信心度檢查] 答案: 您好，我是信用卡部門人員，正在處理您的信用卡問題。 → 信心度=0.95
您好，我是信用卡部門人員，正在處理您的信用卡問題。


In [7]:
# --- 測試3 ---
print(customer_service("我想問關於房貸利率"))

[分類] 問題: 我想問關於房貸利率 → 部門: human (重試次數=0)
AI客服無法為您解決問題，已轉人工客服處理。
